# Stock Price Classification
By: Jared Berry

In [2]:
import warnings
warnings.filterwarnings('always') # To deal with 'UndefinedMetric' warnings

In [3]:
# I/O and data structures
import pickle
import pandas as pd
import numpy as np

# Classification models
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor

# Model selection
from sklearn.model_selection import KFold
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV

# Evaluation
from sklearn import metrics

# Quality of life
import os
import time

C:\Users\jared\Anaconda3\lib\site-packages\lightgbm\__init__.py:30: ResourceWarning: unclosed file <_io.TextIOWrapper name='C:\\Users\\jared\\Anaconda3\\lib\\site-packages\\lightgbm\\VERSION.txt' mode='r' encoding='cp1252'>
  __version__ = open(os.path.join(dir_path, 'VERSION.txt')).read().strip()


## Set-up

#### Imports

In [4]:
# Import modeling helper functions
from modeling_funcs import *

In [5]:
# Import
inpath = "model_dictionary.pickle"
with open(inpath, 'rb') as f:
    modeling = pickle.load(f)

In [6]:
modeling.keys()

dict_keys(['target_1_return', 'target_1_return_res', 'target_1_composite', 'target_1_average', 'target_1_rank', 'target_1_up', 'target_1_rel_return', 'target_5_return', 'target_5_return_res', 'target_5_composite', 'target_5_average', 'target_5_rank', 'target_5_up', 'target_5_rel_return', 'target_10_return', 'target_10_return_res', 'target_10_composite', 'target_10_average', 'target_10_rank', 'target_10_up', 'target_10_rel_return', 'target_21_return', 'target_21_return_res', 'target_21_composite', 'target_21_average', 'target_21_rank', 'target_21_up', 'target_21_rel_return', 'features', 'ticker_features'])

In [7]:
# Pull out the features dataframe
train = modeling['features']

#### Feature selection

In [10]:
# Set a feature selection list (THINK ABOUT INFORMING THIS SELECTION WITH SHRINKAGE METHODS, I.E. RIDGE REGRESSION)
features = ['High', 'Low', 'Open', 'Close', 'Volume', 'AdjClose', 'Year',
            'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear', 'Pct_Change_Daily',
            'Pct_Change_Monthly', 'Pct_Change_Yearly', 'RSI', 'Volatility',
            'Yearly_Return_Rank', 'Monthly_Return_Rank', 'Pct_Change_Class',
            'Rolling_Yearly_Mean_Positive_Days', 'Rolling_Monthly_Mean_Positive_Days', 
            'Rolling_Monthly_Mean_Price', 'Rolling_Yearly_Mean_Price',
            'Momentum_Quality_Monthly', 'Momentum_Quality_Yearly', 'SPY_Trailing_Month_Return',
            'open_l1', 'open_l5', 'open_l10', 'close_l1', 'close_l5', 'close_l10',
            'return_prev1_open_raw', 'return_prev5_open_raw', 'return_prev10_open_raw',
            'return_prev1_close_raw', 'return_prev5_close_raw', 'return_prev10_close_raw',
            'pe_ratio', 'debt_ratio', 'debt_to_equity', 'roa',
            'beta']

In [11]:
# Select on features to pass to modeling machinery, along with necessary indexers
X = train[features]
tickers = train['ticker'].unique().tolist()

In [8]:
# Choose a ticker
target = modeling['target_21_rel_return']

## Modeling

#### Panel-level

Given that there are bound to be a number of systemic considerations that impact the price of a stock at any given point in time, it is prudent to perform and evaluate predictions across the panel of S&P 500 stocks in our sample, which will capture potential linkages between different stocks, and allow us to explore the possibility of using features generated from clustering to group like stocks in the panel.

In [18]:
# Create a panel-level copy
y_p = target.copy()

# Indexes of hold-out test data (the 21 days of data preceding the present day)
test_idx = np.where(np.isnan(y_p))[0].tolist()

# In order to ensure grouping is done properly, remove this data from a ticker-identification set as well
ticker_locs = train['ticker'].drop(train.index[test_idx]).reset_index().drop('index', axis=1)

In [19]:
# Create a panel-level copy
X_p = X.copy(deep=True)

# Simple feature-scaling - for now, replace missings with 0 (i.e. the mean of a normalized feature) within days
X_p = X_p.groupby(['Year', 'Month', 'Day']).apply(lambda x: (x - np.mean(x))/np.std(x)).fillna(0)

In [20]:
# Remove hold-out test data
y_p = np.delete(y_p, test_idx)
X_p_holdout = X_p.loc[X_p.index[test_idx]]
X_p = X_p.drop(X_p.index[test_idx])

In [26]:
y_p_smoothed = np.zeros(y_p.shape[0])
for t in tickers:
    idx = ticker_locs.loc[ticker_locs['ticker'] == t].index.tolist()
    y_to_smooth = y_p[idx]
    
    # Compute EMA smoothing of target within ticker
    EMA = 0
    gamma_ = 0.75
    for ti in range(len(y_to_smooth)):
        EMA = gamma_*y_to_smooth[ti] + (1-gamma_)*EMA
        y_to_smooth[ti] = EMA
        
    y_p_smoothed[idx] = y_to_smooth

In [24]:
y_p_smoothed = y_p.copy()

In [ ]:
# Fit and evaluate - gamma MUST be 1 here
## fit_lgbm_classifier(X_p, y_p_smoothed, X_p_holdout, ticker="", ema_gamma=1, n_splits=12, export=False, 
##                     valid_method = 'panel', groups = ticker_locs, labeled = False)
fit_sklearn_classifier(X_p, y_p_smoothed, X_p_holdout, ticker="", ema_gamma=1, n_splits=12, model=KNeighborsClassifier,
                       label='kNN Classifier', param_search = {}, export=False, labeled=False, groups=ticker_locs,
                       cv_method='panel'
                      )


0 targets changed by smoothing.
Baseline, one-class accuracy is: 50.0%
Classification report for one-class predictor:


C:\Users\jared\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\jared\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\jared\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00    490499
           1       0.50      1.00      0.67    495151

   micro avg       0.50      0.50      0.50    985650
   macro avg       0.25      0.50      0.33    985650
weighted avg       0.25      0.50      0.34    985650

Baseline, random-walk accuracy is: 61.0%
Classification report for random-walk predictor:
              precision    recall  f1-score   support

           0       0.61      0.61      0.61    490235
           1       0.61      0.61      0.61    494911

   micro avg       0.61      0.61      0.61    985146
   macro avg       0.61      0.61      0.61    985146
weighted avg       0.61      0.61      0.61    985146

Training model on validation split #1


#### Ticker-level 

At the heart of this analysis is a time-series prediction problem. As such, it is prudent to explore running models for each individual stock. We can envision averaging the results of both modeling approaches to incorporate the contribution of both into a final prediction.

In [17]:
for i, t in enumerate(tickers[:5]):
    
    # Pull only feature/target data for the relevant stocker
    X_t = X.loc[train['ticker'] == t,:]
    y_t = np.array(target)[train['ticker'] == t]
    
    # Indexes of hold-out test data (the 21 days of data preceding the present day)
    test_idx = np.where(np.isnan(y_t))[0].tolist()
    
    # Simple feature-scaling - for now, replace missings with 0 (i.e. the mean of a normalized feature)
    X_t = X_t.apply(lambda x: (x - np.mean(x))/np.std(x)).fillna(0)
    
    # Remove hold-out test data
    y_t = np.delete(y_t, test_idx)
    X_t_holdout = X_t.loc[X_t.index[test_idx]]
    X_t = X_t.drop(X_t.index[test_idx])
    
    # Fit and evaluate
    #fit_lgbm_classifier(X_t, y_t, X_t_holdout, ticker=t, ema_gamma=1, valid_splits=12, export=False, valid_method='ts', labeled = False)
    fit_sklearn_classifier(X_t, y_t, X_t_holdout, ticker=t, ema_gamma=1, n_splits=12, model=KNeighborsClassifier,
                           label='kNN Classifier', param_search = {'n_neighbors':[2,4,6,8,10]}, 
                          )


0 targets changed by smoothing.
Baseline, one-class accuracy is: 56.00000000000001%
Classification report for one-class predictor:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       885
           1       0.56      1.00      0.72      1130

   micro avg       0.56      0.56      0.56      2015
   macro avg       0.28      0.50      0.36      2015
weighted avg       0.31      0.56      0.40      2015

Baseline, random-walk accuracy is: 48.0%
Classification report for random-walk predictor:
              precision    recall  f1-score   support

           0       0.41      0.41      0.41       885
           1       0.54      0.54      0.54      1129

   micro avg       0.48      0.48      0.48      2014
   macro avg       0.47      0.47      0.47      2014
weighted avg       0.48      0.48      0.48      2014

Performing grid search for hyperparameter tuning


C:\Users\jared\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\jared\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\jared\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Training model on validation split #1
Training model on validation split #2
Training model on validation split #3
Training model on validation split #4
Training model on validation split #5
Training model on validation split #6
Training model on validation split #7
Training model on validation split #8
Training model on validation split #9
Training model on validation split #10
Training model on validation split #11
Training model on validation split #12
Build, hyperparameter selection, and validation of kNN Classifier took 16.511 seconds

Hyperparameters are as follows:
n_neighbors: 10

Validation scores are as follows:
precision    0.512573
recall       0.529570
accuracy     0.529570
f1           0.485275
dtype: float64

0 targets changed by smoothing.
Baseline, one-class accuracy is: 59.0%
Classification report for one-class predictor:


C:\Users\jared\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\jared\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\jared\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       828
           1       0.59      1.00      0.74      1187

   micro avg       0.59      0.59      0.59      2015
   macro avg       0.29      0.50      0.37      2015
weighted avg       0.35      0.59      0.44      2015

Baseline, random-walk accuracy is: 51.0%
Classification report for random-walk predictor:
              precision    recall  f1-score   support

           0       0.41      0.41      0.41       828
           1       0.59      0.59      0.59      1186

   micro avg       0.51      0.51      0.51      2014
   macro avg       0.50      0.50      0.50      2014
weighted avg       0.51      0.51      0.51      2014

Performing grid search for hyperparameter tuning
Training model on validation split #1
Training model on validation split #2
Training model on validation split #3
Training model on validation split #4
Training model on validation split #5
Training model on

...

Training model on validation split #1
Training model on validation split #2
Training model on validation split #3
Training model on validation split #4
Training model on validation split #5
Training model on validation split #6
Training model on validation split #7
Training model on validation split #8
Training model on validation split #9
Training model on validation split #10
Training model on validation split #11
Training model on validation split #12
Build, hyperparameter selection, and validation of kNN Classifier took 16.988 seconds

Hyperparameters are as follows:
n_neighbors: 10

Validation scores are as follows:
precision    0.504884
recall       0.561828
accuracy     0.561828
f1           0.496860
dtype: float64

0 targets changed by smoothing.
Baseline, one-class accuracy is: 56.99999999999999%
Classification report for one-class predictor:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       875
           1       0.57    

C:\Users\jared\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\jared\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\jared\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Training model on validation split #1
Training model on validation split #2
Training model on validation split #3
Training model on validation split #4
Training model on validation split #5
Training model on validation split #6
Training model on validation split #7
Training model on validation split #8
Training model on validation split #9
Training model on validation split #10
Training model on validation split #11
Training model on validation split #12
Build, hyperparameter selection, and validation of kNN Classifier took 17.125 seconds

Hyperparameters are as follows:
n_neighbors: 4

Validation scores are as follows:
precision    0.513656
recall       0.538172
accuracy     0.538172
f1           0.486208
dtype: float64

0 targets changed by smoothing.
Baseline, one-class accuracy is: 55.00000000000001%
Classification report for one-class predictor:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       706
           1       0.55     

C:\Users\jared\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\jared\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\jared\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Training model on validation split #1
Training model on validation split #2
Training model on validation split #3
Training model on validation split #4
Training model on validation split #5
Training model on validation split #6
Training model on validation split #7
Training model on validation split #8
Training model on validation split #9
Training model on validation split #10
Training model on validation split #11
Training model on validation split #12
Build, hyperparameter selection, and validation of kNN Classifier took 11.621 seconds

Hyperparameters are as follows:
n_neighbors: 4

Validation scores are as follows:
precision    0.519550
recall       0.537879
accuracy     0.537879
f1           0.494314
dtype: float64
